In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json, os, math

In [ ]:
models = [
    ("<path>/Experiments/prostate_m3d_M3dSegmentation/", "M3D-NCA"),
    ("<path>/Experiments/prostatefAbl_none_10_1.0_16_OctreeNCASegmentation/", "OctreeNCA"),
    ("<path>/Experiments/prostate_unet_UNetSegmentation/", "UNet"),
    ("<path>/Experiments/prostate_munet3d_UNet_resnet18_0_minUNet3DSegmentation/", "Resnet18"),
    ("<path>/Experiments/prostate_munet3d_UNet_densenet121_0_minUNet3DSegmentation/", "Densenet"),
]

In [ ]:
annnotation_list = []
annotation_dict = {}
all_files = []
for i, (model, model_name) in enumerate(models):
    score = pd.read_csv(f"{model}eval/standard.csv", sep='\t').loc[:, "DiceScore/0"] * 100

    score_std = score.std()
    score = score.mean()

    mem_allocation = json.load(open(f"{model}/mem_allocation.json"))["byte"]
    num_params = json.load(open(f"{model}/num_params.json"))["num_params"]

    eval_file = {}

    eval_file["model"] = model
    eval_file["mem_allocation"] = mem_allocation
    eval_file["num_params"] = num_params
    eval_file["score"] = score
    eval_file["score_std"] = score_std
    df = pd.DataFrame(index=[i], data=eval_file)
    all_files.append(df)
    annnotation_list.append((num_params, score, model_name))
    annotation_dict[model_name] = (num_params, score)

all_files = pd.concat(all_files)

In [ ]:
def get_nnUNet_results(path: str, trainer: str) -> dict:
    FOLD = 0
    TRAIN_TASK = "Task506_Prostate_MEDSeg_split"
    EVAL_TASK = "Task508_Prostate_MEDSeg_split" 
    #both are the same dataset. However 506 is not properly preprocessed, so it cannot be used for evaluation
    # Later I preprocessed 508 correctly, which can also be used for evaluation. Future work should use 508 for training and evaluation

    eval_path = os.path.join(path,"evaluation", "nnUNet_ext", "3d_fullres",
                             TRAIN_TASK, TRAIN_TASK, f"{trainer}__nnUNetPlansv2.1", 
                             f"Generic_UNet/SEQ/head_None/fold_{FOLD}",EVAL_TASK,
                             "val_metrics_all.csv")
    results_path = os.path.join(path, "results", "nnUNet_ext", "3d_fullres",
                             TRAIN_TASK, TRAIN_TASK, f"{trainer}__nnUNetPlansv2.1", 
                             f"Generic_UNet/SEQ/fold_{FOLD}")
    results = pd.read_csv(eval_path, sep='\t')
    results = results[results["split"] == "test"]
    results = results[results["seg_mask"] == "mask_1"]
    results = results[results["metric"] == "Dice"]

    model_name = {
        "nnUNetTrainerNCA": "nnNCA",
        "nnUNetTrainerSequential": "nnUNet",
    }
    ret = {
        "model": model_name[trainer],
        "mem_allocation": json.load(open(os.path.join(results_path, "mem_allocation.json")))["byte"],
        "num_params": json.load(open(os.path.join(results_path, "num_params.json")))["num_params"],
        "score": results["value"].mean() * 100,
        "score_std": (results["value"] * 100).std()
    }
    return ret

In [ ]:
nnOctreeNCA = get_nnUNet_results("<path>/nnunet_nca", "nnUNetTrainerNCA")
nnUNet = get_nnUNet_results("<path>/nnunet_nca", "nnUNetTrainerSequential")
data = [nnUNet]
df = pd.DataFrame(data=data)
all_files = pd.concat([all_files, df])
for model_d in data:
    annnotation_list.append((model_d["num_params"], model_d["score"], model_d["model"]))
    annotation_dict[model_d["model"]] = (model_d["num_params"], model_d["score"])

In [ ]:
from IPython.display import display, HTML
display(HTML(all_files.to_html()))

In [ ]:
sns.set_theme()
ax = sns.scatterplot(data=all_files, x="num_params", y="score", hue="mem_allocation", size="mem_allocation",legend=False, sizes=(20, 500))
for x,y,txt in annnotation_list:
    pass
    #ax.annotate(txt, (x,y), textcoords="offset points", xytext=(0,10), ha='center')


def my_annotate(name, xytext):
    ax.annotate(name, annotation_dict[name], textcoords="offset points", xytext=xytext, ha='center',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"),
        arrowprops=dict(arrowstyle="->", color='black'))

my_annotate("M3D-NCA", (30, -30))
my_annotate("OctreeNCA", (30, 30))
my_annotate("Resnet18", (-20, -70))
my_annotate("nnUNet", (-30, 20))
my_annotate("UNet", (-40, 10))
my_annotate("Densenet", (-40, -40))

plt.xscale("log")
plt.xlabel("Number of parameters")
plt.ylabel("Dice")
plt.title("Radiology segmentation")
plt.ylim(40, 100)
plt.savefig("figures/prostate_segmentation.pdf", bbox_inches='tight')

In [ ]:
min_vram = math.floor(10 * all_files["mem_allocation"].min() / 1024**3) / 10
max_vram = math.ceil(10 * all_files["mem_allocation"].max() / 1024**3) / 10


def compute_circle_size(mem_allocation):
    mem_allocation = mem_allocation / 1024**3
    return 20 + 480 * (mem_allocation - min_vram) / (max_vram - min_vram)

In [ ]:
all_files.loc[:, "circle_size"] = all_files["mem_allocation"].apply(compute_circle_size)

In [ ]:
sns.set_theme()
ax = sns.scatterplot(data=all_files, x="num_params", y="score", hue="mem_allocation", size="mem_allocation",legend=False, sizes=(20, 500))


def my_annotate(name, xytext):
    ax.annotate(name, annotation_dict[name], textcoords="offset points", xytext=xytext, ha='center',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"),
        arrowprops=dict(arrowstyle="->", color='black'))

my_annotate("M3D-NCA", (30, -30))
my_annotate("OctreeNCA", (30, 30))
my_annotate("Resnet18", (-20, -70))
my_annotate("nnUNet", (-30, 20))
my_annotate("UNet", (-40, 10))
my_annotate("Densenet", (-40, -40))

size_values = [20, 500]
size_labels = [f"{min_vram} GiB", f"{max_vram} GiB"]
handles = [
    plt.scatter([], [], s=size, edgecolor="black", color="grey", label=label)
    for size, label in zip(size_values, size_labels)  # Adjust sizes to match your plot
]
# Add the custom legend
plt.legend(
    handles=handles,
    title="Memory Allocation",
    labelspacing=1.2,
    loc="lower left",
    borderpad=1.0,
    frameon=True
)

plt.xscale("log")
plt.xlabel("Number of parameters")
plt.ylabel("Dice")
plt.title("Radiology segmentation")
plt.ylim(40, 100)
plt.savefig("figures/prostate_segmentation.pdf", bbox_inches='tight')